In [26]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import random
from googletrans import Translator


In [27]:
med = Translator().translate('acido acetilsalicilico', src='es', dest='en').text.replace(' ', '+')
med

'acetylsalicylic+acid'

In [28]:
url='https://www.pharmacychecker.com/'+med+'/?src=drug-suggest#prices'
url

'https://www.pharmacychecker.com/acetylsalicylic+acid/?src=drug-suggest#prices'

In [29]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--disable-cache')
options.add_argument('--disable-cookies')

In [31]:
# #Para genéricos
driver = webdriver.Chrome(service=Service("C:/chromedriver.exe"),options=options)
# Para genéricos
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
results_generics = soup.find_all("li", {"class": "drug-pricing-item mt-2"})

# Para patentes
try:
    element = driver.find_element(By.CLASS_NAME, 'drug-type-link')
    driver.execute_script("arguments[0].click();", element)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    results_patents = soup.find_all("li", {"class": "drug-pricing-item mt-2"})
except:
    results_patents = []
    driver.quit()

In [32]:
print("elementos genéricos: ", len(results_generics))
print("elementos patente: ",len(results_patents))

elementos genéricos:  4
elementos patente:  0


In [ ]:
results_patents

[<li class="drug-pricing-item mt-2" data-sort-order="0">
 <div class="row hide-on-mobile mt-3">
 <div class="col-lg-9">
 <div class="row">
 <div class="col-6">
 <a class="cpc-drug-listings-link cpc-drug-listings-link-image recaptcha-link" data-calt="125" data-drug="Nexavar" data-pharm="affordablerxmeds.com" data-recaptcha-action="cpc_drug_listings_link" href="https://www.pharmacychecker.com/cpcdrugclick/?guid=9ecf8427-41de-4a5b-99fd-4285786ca699&amp;drugid=72499&amp;drugstrengthid=134965&amp;drugquantityid=382230&amp;position=1&amp;cpc-click-hash=i3aubunw9" rel="nofollow" target="_blank"><img alt="AffordableRxMeds.com" height="72" loading="lazy" src="https://cdn.pharmacychecker.com/images/logos/1209_border.png" title="" width="180"/></a>
 <a class="hide-on-mobile" data-cy="pharmacy-profile" href="/pharmacies/affordablerxmeds.com/1209/"><i class="fa fa-info-circle pharmacy-profile-tooltippharmacy-profile-tooltip" data-original-title="Click to see pharmacy profile" data-toggle="tooltip" 

In [ ]:
# if not results:
#     df=pd.DataFrame()  
# else:
#     data=[]
#     for result in results:
#         info={}
#         info["precio"]=result.find("p",{"class":"desktop-price"}).text
#         info["farmacia"] = result.find("a", {"class": "cpc-drug-listings-link-image"}).get("data-pharm")
#         info["producto"] = result.find("a", {"class": "cpc-drug-listings-link-image"}).get("data-drug")
#         info["cantidad"] = result.find("div", {"class": "col-3 col-lg-3 drug-pricing-details hide-on-mobile"}).text
#         #Añadir país
#         data.append(info)
#     df=pd.DataFrame(data)
#     #Añadir pais
#     canada_pharmacies = ['liferxpharmacy', 'canadianprescriptiondrugstore', 'medsengage', 'offshorecheapmeds', 'pricepropharmacy']
#     df["pais"] = df["farmacia"].apply(lambda x: "Canada" if any([pharmacy in x for pharmacy in canada_pharmacies]) else "Estados Unidos")